In [ ]:
from CompactFIPS202 import *
import binascii
import os
from hashlib import sha3_256, sha3_512, shake_128, shake_256
from kyber import Kyber512, Kyber768, Kyber1024
from aes256_ctr_drbg import AES256_CTR_DRBG
import itertools

os.system('rm -rf ./vec')

def parse_kat_data(data):
    parsed_data = {}
    count_blocks = data.split('\n\n')
    for block in count_blocks[1:-1]:
        block_data = block.split('\n')
        count, seed, pk, sk, ct, ss = [line.split(" = ")[-1] for line in block_data]
        parsed_data[count] = {
            "seed": bytes.fromhex(seed),
            "pk": bytes.fromhex(pk),
            "sk": bytes.fromhex(sk),
            "ct": bytes.fromhex(ct),
            "ss": bytes.fromhex(ss),   
        }
    return parsed_data

In [ ]:
with open("assets/PQCkemKAT_1632.rsp") as f:
    kat_data = f.read()
    parsed_data = parse_kat_data(kat_data)

data = parsed_data['0']
seed, pk, sk, ct, ss = data.values()
        
# Seed DRBG with KAT seed
Kyber512.set_drbg_seed(seed)
# Assert keygen matches
_pk, _sk = Kyber512.keygen()

# Assert encapsulation matches
_ct, _ss = Kyber512.enc(_pk)

#    # Assert decapsulation matches
#    __ss = Kyber512.dec(ct, sk)

In [ ]:
seed.hex()

In [ ]:
(pk == _pk) == (sk == _sk) == (ct == _ct) == (ss == _ss)

In [ ]:
random_bytes = os.urandom

In [ ]:
a = random_bytes(32)

In [ ]:
Kyber512._xof_a.hex()

In [ ]:
a = shake_128(Kyber512._xof_input_bytes).digest(Kyber512._xof_length).hex()

In [ ]:
b = SHAKE128(Kyber512._xof_input_bytes, Kyber512._xof_length).hex()

In [ ]:
a == b

In [ ]:
a

In [ ]:
len (a)

In [ ]:
b

In [ ]:
a = sha3_256(Kyber512._xof_input_bytes).digest().hex()

In [ ]:
b = SHA3_256(Kyber512._xof_input_bytes).hex()

In [ ]:
a == b

In [ ]:
a

In [ ]:
b

In [ ]:
rates = [1344, 1088, 1152, 1088, 832, 579]

In [ ]:
for rate in rates:
    print(rate>>3)

In [ ]:
512//8

In [ ]:
a = bytearray([0 for i in range(2)])
b = bytearray([0 for i in range(2)])
c = bytearray(2)
c[0] = a[0] ^ b[0]
c[0]

In [ ]:
72*8/64